# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/sales_train_validation.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Parameters settings

In [7]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100

In [8]:
validation_training_ratio = 0.99

In [9]:
list_params = list()
for p in [0, 1, 2, 3]:
    for d in range(2):
        for q in range(4):
            list_params.append((p, d, q))

In [10]:
list_params

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 1, 0),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3)]

In [11]:
len(list_params)

32

# This step involves specifying Demand Pattern (Intermittent or Lumpy or Erratic or Smooth)

In [12]:
sales_pattern = sales_intermittent.copy()

In [13]:
sales_pattern_py = sales_pattern.copy()
list_pattern_py = sales_pattern_py.id.unique().tolist()

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
test_data = df_pattern.iloc[n_pred_days*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

# Hyperparameter for ARIMA model

In [14]:
list_sample = sample(list_pattern_py, 50)

In [15]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in list_sample:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_sample.index(product) / len(list_sample) * 100, 2)))
    train_first = train_data_first_part[product].iloc[-num_train_needed:]
    train_second = train_data_second_part[product]
    
    for combi in list_params:
        model = ARIMA(train_first, order = combi)
        model_fit = model.fit()
        forecast_temp = model_fit.forecast(len(train_second))
        df_temp = pd.DataFrame({'Date': train_second.index,
                                'Actual Data': train_second.values,
                                'Forecast': forecast_temp.values,
                                'Combination': [combi for combi_count in range(len(train_second))], 
                                'Product': [product for product_count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_temp, ignore_index=False)
        
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_2_364_CA_4_validation
Progressing: 0.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_2_073_CA_1_validation
Progressing: 2.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimiz

Currently Running Product: FOODS_2_015_CA_3_validation
Progressing: 4.0 %


...

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_2_072_CA_1_validation
Progressing: 98.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


This line of code took 4.4998647755166665 minutes


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [16]:
df_result_all_params

,Date,Actual Data,Forecast,Combination,Product
0,2016-04-03,0.0,0.639995,"(0, 0, 0)",FOODS_2_364_CA_4_validation
1,2016-04-04,0.0,0.639995,"(0, 0, 0)",FOODS_2_364_CA_4_validation
2,2016-04-05,2.0,0.639995,"(0, 0, 0)",FOODS_2_364_CA_4_validation
3,2016-04-06,0.0,0.639995,"(0, 0, 0)",FOODS_2_364_CA_4_validation
4,2016-04-07,0.0,0.639995,"(0, 0, 0)",FOODS_2_364_CA_4_validation
...,...,...,...,...,...
3,2016-04-06,0.0,0.402139,"(3, 1, 3)",FOODS_2_072_CA_1_validation
4,2016-04-07,0.0,0.708098,"(3, 1, 3)",FOODS_2_072_CA_1_validation
5,2016-04-08,0.0,0.721379,"(3, 1, 3)",FOODS_2_072_CA_1_validation
6,2016-04-09,0.0,0.399445,"(3, 1, 3)",FOODS_2_072_CA_1_validation


# Summay all metrics

In [17]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [18]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Combination']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0742513784000001 minutes


In [19]:
df_result_metrics_all_params

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_022_CA_2_validation,"(0, 0, 0)",0.717496,163.9990,175.7579,68.0005,0.409997,0.438633
1,FOODS_1_022_CA_2_validation,"(0, 0, 1)",0.717894,164.0901,175.7559,67.9971,0.410225,0.438793
2,FOODS_1_022_CA_2_validation,"(0, 0, 2)",0.721745,164.9703,175.7371,67.9643,0.412426,0.440352
3,FOODS_1_022_CA_2_validation,"(0, 0, 3)",0.710111,162.3110,175.7910,68.0583,0.405778,0.435723
4,FOODS_1_022_CA_2_validation,"(0, 1, 0)",0.437500,100.0000,NaN,100.0000,0.250000,0.500000
...,...,...,...,...,...,...,...,...
1595,FOODS_3_802_CA_2_validation,"(3, 0, 3)",0.985819,168.9975,165.0598,46.2035,0.422494,0.429175
1596,FOODS_3_802_CA_2_validation,"(3, 1, 0)",1.035649,177.5398,167.1223,49.7430,0.443850,0.463768
1597,FOODS_3_802_CA_2_validation,"(3, 1, 1)",0.984397,168.7538,171.5189,60.0568,0.421884,0.436987
1598,FOODS_3_802_CA_2_validation,"(3, 1, 2)",0.979102,167.8461,171.1679,58.8999,0.419615,0.441369


# Check what products has unexpected metrics result

In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_1_156_CA_1_validation,"(0, 0, 0)",inf,inf,200.0,NaN,0.369995,0.369995
161,FOODS_1_156_CA_1_validation,"(0, 0, 1)",inf,inf,200.0,NaN,0.363861,0.364116
162,FOODS_1_156_CA_1_validation,"(0, 0, 2)",inf,inf,200.0,NaN,0.366953,0.367136
163,FOODS_1_156_CA_1_validation,"(0, 0, 3)",inf,inf,200.0,NaN,0.352130,0.352835
164,FOODS_1_156_CA_1_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1563,FOODS_3_776_CA_2_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1564,FOODS_3_776_CA_2_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1565,FOODS_3_776_CA_2_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1566,FOODS_3_776_CA_2_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_1_156_CA_1_validation,"(0, 0, 0)",inf,inf,200.0,NaN,0.369995,0.369995
161,FOODS_1_156_CA_1_validation,"(0, 0, 1)",inf,inf,200.0,NaN,0.363861,0.364116
162,FOODS_1_156_CA_1_validation,"(0, 0, 2)",inf,inf,200.0,NaN,0.366953,0.367136
163,FOODS_1_156_CA_1_validation,"(0, 0, 3)",inf,inf,200.0,NaN,0.352130,0.352835
164,FOODS_1_156_CA_1_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1563,FOODS_3_776_CA_2_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1564,FOODS_3_776_CA_2_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1565,FOODS_3_776_CA_2_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1566,FOODS_3_776_CA_2_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4,FOODS_1_022_CA_2_validation,"(0, 1, 0)",0.4375,100.0,NaN,100.0,0.250,0.500000
68,FOODS_1_064_CA_4_validation,"(0, 1, 0)",0.8750,100.0,NaN,100.0,0.500,0.707107
100,FOODS_1_081_CA_1_validation,"(0, 1, 0)",0.4375,100.0,NaN,100.0,1.125,2.371708
164,FOODS_1_156_CA_1_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000,0.000000
196,FOODS_2_006_CA_4_validation,"(0, 1, 0)",0.4375,100.0,NaN,100.0,0.250,0.500000
...,...,...,...,...,...,...,...,...
1559,FOODS_3_776_CA_2_validation,"(2, 1, 3)",NaN,NaN,NaN,NaN,0.000,0.000000
1564,FOODS_3_776_CA_2_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000,0.000000
1565,FOODS_3_776_CA_2_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000,0.000000
1566,FOODS_3_776_CA_2_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000,0.000000


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_1_156_CA_1_validation,"(0, 0, 0)",inf,inf,200.0,NaN,0.369995,0.369995
161,FOODS_1_156_CA_1_validation,"(0, 0, 1)",inf,inf,200.0,NaN,0.363861,0.364116
162,FOODS_1_156_CA_1_validation,"(0, 0, 2)",inf,inf,200.0,NaN,0.366953,0.367136
163,FOODS_1_156_CA_1_validation,"(0, 0, 3)",inf,inf,200.0,NaN,0.352130,0.352835
164,FOODS_1_156_CA_1_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1563,FOODS_3_776_CA_2_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1564,FOODS_3_776_CA_2_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1565,FOODS_3_776_CA_2_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1566,FOODS_3_776_CA_2_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [26]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [27]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 36.0 %
Percentage of unexpected values of WMAPE is: 36.0 %
Percentage of unexpected values of SMAPE is: 52.0 %
Percentage of unexpected values of MAPE is: 36.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [28]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Combination

In [29]:
df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Combination,,,,,,
"(0, 0, 0)",0.862285,113.663387,133.241378,56.055069,1.462031,1.810508
"(0, 0, 1)",0.864430,113.763484,133.391581,56.249703,1.467457,1.817116
"(0, 0, 2)",0.866761,113.711806,133.569275,56.435063,1.468309,1.819297
"(0, 0, 3)",0.856631,112.618216,133.585400,56.337375,1.465012,1.812594
"(0, 1, 0)",0.897053,100.811741,130.644677,67.851841,1.863636,2.360173
"(0, 1, 1)",0.860632,111.150087,125.198390,51.419316,1.482537,1.813807
"(0, 1, 2)",0.864157,111.789890,123.755300,50.497335,1.483673,1.815951
"(0, 1, 3)",0.874053,112.845255,125.234310,51.008565,1.498196,1.829219
"(1, 0, 0)",0.863635,113.621409,133.424597,56.301353,1.467505,1.817139


In [30]:
for mts in list_metrics:
    print('The optimum Combination based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Combination based on Mean MASE is: (1, 1, 0)
The optimum Combination based on Mean WMAPE is: (1, 1, 0)
The optimum Combination based on Mean SMAPE is: (3, 1, 1)
The optimum Combination based on Mean MAPE is: (3, 1, 1)
The optimum Combination based on Mean MAE is: (0, 0, 0)
The optimum Combination based on Mean RMSE is: (3, 0, 0)


### Get MEDIAN metrics value of each Combination

In [31]:
df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Combination,,,,,,
"(0, 0, 0)",0.794869,100.00000,137.49860,57.27785,0.869998,1.076652
"(0, 0, 1)",0.797419,100.41650,137.73015,57.22600,0.864642,1.074003
"(0, 0, 2)",0.836222,100.40240,137.64485,57.07005,0.854187,1.059964
"(0, 0, 3)",0.820261,100.99040,135.90080,55.94485,0.857988,1.053589
"(0, 1, 0)",0.875000,100.00000,129.16665,62.42060,0.937500,1.224745
"(0, 1, 1)",0.834283,100.00000,129.08340,51.23550,0.801398,1.030517
"(0, 1, 2)",0.746994,100.42800,129.07210,49.24290,0.805995,1.032896
"(0, 1, 3)",0.846902,100.46320,128.59310,49.29200,0.755201,1.012753
"(1, 0, 0)",0.800778,100.39735,137.72320,57.35970,0.859087,1.076777


In [32]:
for mts in list_metrics:
    print('The optimum Combination based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Combination based on Median of MASE is: (1, 1, 0)
The optimum Combination based on Median of WMAPE is: (2, 1, 0)
The optimum Combination based on Median of SMAPE is: (0, 1, 3)
The optimum Combination based on Median of MAPE is: (3, 1, 1)
The optimum Combination based on Median of MAE is: (0, 1, 3)
The optimum Combination based on Median of RMSE is: (0, 1, 3)


# Run ARIMA model after decide best paramaters

In [33]:
best_combination = (0, 1, 3)

In [34]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    train_product = train_data[product].iloc[num_train_needed*-1:]
    test_product = test_data[product]  
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()

    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Date': test_product.index,
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values,
                                   'Product': [product for product_count in range(len(test_product))]})
    
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_001_CA_1_validation
Progressing: 0.0 %
Currently Running Product: FOODS_1_002_CA_1_validation
Progressing: 0.03 %
Currently Running Product: FOODS_1_003_CA_1_validation
Progressing: 0.06 %
Currently Running Product: FOODS_1_006_CA_1_validation
Progressing: 0.08 %
Currently Running Product: FOODS_1_008_CA_1_validation
Progressing: 0.11 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_1_009_CA_1_validation
Progressing: 0.14 %
Currently Running Product: FOODS_1_010_CA_1_validation
Progressing: 0.17 %
Currently Running Product: FOODS_1_011_CA_1_validation
Progressing: 0.2 %
Currently Running Product: FOODS_1_015_CA_1_validation
Progressing: 0.22 %
Currently Running Product: FOODS_1_016_CA_1_validation
Progressing: 0.25 %
Currently Running Product: FOODS_1_020_CA_1_validation
Progressing: 0.28 %
Currently Running Product: FOODS_1_022_CA_1_validation
Progressing: 0.31 %
Currently Running Product: FOODS_1_023_CA_1_validation
Progressing: 0.34 %
Currently Running Product: FOODS_1_024_CA_1_validation
Progressing: 0.36 %
Currently Running Product: FOODS_1_025_CA_1_validation
Progressing: 0.39 %
Currently Running Product: FOODS_1_026_CA_1_validation
Progressing: 0.42 %
Currently Running Product: FOODS_1_028_CA_1_validation
Progressing: 0.45 %
Currently Running Product: FOODS_1_029_CA_1_validation
Progressing: 0.47 %
Currently Running Product:

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_1_052_CA_1_validation
Progressing: 0.81 %
Currently Running Product: FOODS_1_053_CA_1_validation
Progressing: 0.84 %
Currently Running Product: FOODS_1_054_CA_1_validation
Progressing: 0.87 %
Currently Running Product: FOODS_1_056_CA_1_validation
Progressing: 0.89 %
Currently Running Product: FOODS_1_057_CA_1_validation
Progressing: 0.92 %


...

Currently Running Product: FOODS_3_798_CA_4_validation
Progressing: 99.55 %
Currently Running Product: FOODS_3_799_CA_4_validation
Progressing: 99.58 %
Currently Running Product: FOODS_3_800_CA_4_validation
Progressing: 99.61 %
Currently Running Product: FOODS_3_802_CA_4_validation
Progressing: 99.64 %
Currently Running Product: FOODS_3_806_CA_4_validation
Progressing: 99.66 %
Currently Running Product: FOODS_3_810_CA_4_validation
Progressing: 99.69 %
Currently Running Product: FOODS_3_812_CA_4_validation
Progressing: 99.72 %
Currently Running Product: FOODS_3_813_CA_4_validation
Progressing: 99.75 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_815_CA_4_validation
Progressing: 99.78 %
Currently Running Product: FOODS_3_817_CA_4_validation
Progressing: 99.8 %
Currently Running Product: FOODS_3_820_CA_4_validation
Progressing: 99.83 %
Currently Running Product: FOODS_3_821_CA_4_validation
Progressing: 99.86 %
Currently Running Product: FOODS_3_823_CA_4_validation
Progressing: 99.89 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_824_CA_4_validation
Progressing: 99.92 %
Currently Running Product: FOODS_3_825_CA_4_validation
Progressing: 99.94 %
Currently Running Product: FOODS_3_826_CA_4_validation
Progressing: 99.97 %
This line of code took 9.083830151016665 minutes


In [35]:
df_result_best_params

,Date,Actual Data,Forecast,Product
0,2016-04-11,0.0,0.739242,FOODS_1_001_CA_1_validation
1,2016-04-12,4.0,0.449539,FOODS_1_001_CA_1_validation
2,2016-04-13,2.0,0.675689,FOODS_1_001_CA_1_validation
3,2016-04-14,0.0,0.675689,FOODS_1_001_CA_1_validation
4,2016-04-15,0.0,0.675689,FOODS_1_001_CA_1_validation
...,...,...,...,...
50129,2016-04-20,5.0,1.498251,FOODS_3_826_CA_4_validation
50130,2016-04-21,3.0,1.498251,FOODS_3_826_CA_4_validation
50131,2016-04-22,1.0,1.498251,FOODS_3_826_CA_4_validation
50132,2016-04-23,0.0,1.498251,FOODS_3_826_CA_4_validation


In [36]:
#df_result_best_params.to_csv('ARIMA_Intermittent_Point_Forecast.csv')

In [37]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [38]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1_validation,0.754027,129.9246,159.6415,57.2110,0.928033,1.210745
1,FOODS_1_001_CA_4_validation,0.845866,136.6399,172.9908,67.3723,0.780799,1.303832
2,FOODS_1_002_CA_1_validation,0.654471,140.9630,199.0743,96.6515,0.302064,0.781188
3,FOODS_1_002_CA_3_validation,1.596638,171.9456,182.9492,75.1615,0.368455,0.420136
4,FOODS_1_002_CA_4_validation,0.805803,108.4735,162.3597,70.6783,0.929773,1.181421
...,...,...,...,...,...,...,...
3576,FOODS_3_825_CA_4_validation,3.993354,860.1070,189.0007,37.4050,0.614362,0.619034
3577,FOODS_3_826_CA_1_validation,1.009627,95.1380,112.6882,85.4509,2.174582,2.464614
3578,FOODS_3_826_CA_2_validation,0.852986,114.8251,108.5095,15.6720,0.656143,0.812720
3579,FOODS_3_826_CA_3_validation,inf,inf,200.0000,NaN,0.000282,0.000282


In [39]:
df_result_final.RMSE.mean()

1.0248601714916805

In [40]:
df_result_final.MAPE.mean()

60.51469357959543

In [45]:
df_result_final.to_csv('/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/ARIMA/California/Foods/ARIMA_Intermittent_Metrics.csv')